<a href="https://colab.research.google.com/github/SELF-msselve/UTN/blob/main/CEL_T%C3%A9cnicas_de_anonimizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anonimización de datos

La información de identificación personal (PII) es cualquier dato que pueda identificar a una persona específica, como el nombre, el número de identificación emitido por el gobierno, la fecha de nacimiento, la ocupación o la dirección.

La anonimización es una técnica de procesamiento de datos que elimina o modifica la PII. Genera como resultado
datos anónimos que no pueden asociarse a ninguna persona.

A continuación, vamos a ver ejemplos para proteger datos confidenciales y sensibles.


Primero vamos a generar datos falsos, por medio de la librería [faker](https://https://faker.readthedocs.io/en/master/), que contengan datos sensibles o confidenciales.

In [ ]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00


In [ ]:
!mkdir -p datalake/sensitive/personal_data/

In [ ]:
from faker import Faker
import pandas as pd
import random

# Inicializar la instancia de Faker
fake = Faker()

# Crear listas para almacenar los datos generados
num_records = 100  # Número de registros en el dataset
data = []

# Generar datos ficticios sensibles
for _ in range(num_records):
    full_name = fake.name()
    email = fake.email()
    phone_number = fake.msisdn()
    birthdate = fake.date_of_birth(minimum_age=18, maximum_age=80)
    credit_card_number = fake.credit_card_number()
    social_security_number = fake.ssn()

    data.append([full_name, email, phone_number, birthdate, credit_card_number, social_security_number])

# Crear un DataFrame con los datos generados
columns = ['full_name', 'email', 'phone_number', 'birth_date', 'credit_card_number', 'ssn']
df_sensitive = pd.DataFrame(data, columns=columns)

df_sensitive.to_csv("datalake/sensitive/personal_data/people.csv", index=None)

df_sensitive.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Kimberly Burnett,aaronmartin@example.net,5930311660117,1990-08-21,4012296515373,175-01-7801
1,Cheyenne Mendez,josephalejandro@example.com,1887903119443,1945-06-03,4642277088310644,483-80-4893
2,Francisco Williams,rodneygray@example.net,2533578229204,1959-10-08,3585286093919100,091-36-2141
3,Rebecca Phillips,davisbryan@example.org,5013047768596,1972-06-01,4732233372014,754-41-3919
4,Andrea Jenkins,dhodge@example.net,4825387235845,1963-06-05,6560011392199846,114-35-0580


## Enmascaramiento parcial

Consiste en ocultar una porción de los datos con caracteres genéricos. La parte que no se oculta, por lo general, contiene información genérica relevante para análisis. Por ejemplo:

- en el caso de un email, se puede mantener el dominio, lo que está después del @.
- en el caso de un nro. de teléfono, se puede mantener los primeros dígitos para identificar el código de país y de área.

Veamos unos ejemplos.

In [ ]:
def mask_phone_number(df, column_name, n=5):
    """
    Enmascara parcialmente los números de teléfono
    en la columna especificada del DataFrame.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a anonimizar.

    Returns:
        pd.DataFrame: El DataFrame modificado con los números de teléfono enmascarados.
    """
    try:
        # Mantener los primeros 5 dígitos, reemplazar el resto con 'XXX'
        df.loc[:, column_name] = df[column_name].str[:n] + 'XXXXXX'
        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df

def mask_credit_card(df, column_name, n=4):
    """
    Enmascara los números de tarjeta de crédito
    en la columna especificada del DataFrame.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a anonimizar.

    Returns:
        pd.DataFrame: El DataFrame modificado con los números de tarjeta de crédito anonimizados.
    """
    try:
        # Mantener los últimos 4 dígitos, reemplazar el resto con 'XXX'
        df.loc[:, column_name] = 'XXXX-XXXX-' + df[column_name].str[-n:]
        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df

In [ ]:
df_anonymized = df_sensitive.copy()

In [ ]:
#df_anonymized = mask_phone_number(df_anonymized.copy(), "phone_number")
df_anonymized = mask_credit_card(df_anonymized.copy(), "credit_card_number")

In [ ]:
df_anonymized.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Kimberly Burnett,aaronmartin@example.net,59303XXXXXX,1990-08-21,XXXX-XXXX-5373,175-01-7801
1,Cheyenne Mendez,josephalejandro@example.com,18879XXXXXX,1945-06-03,XXXX-XXXX-0644,483-80-4893
2,Francisco Williams,rodneygray@example.net,25335XXXXXX,1959-10-08,XXXX-XXXX-9100,091-36-2141
3,Rebecca Phillips,davisbryan@example.org,50130XXXXXX,1972-06-01,XXXX-XXXX-2014,754-41-3919
4,Andrea Jenkins,dhodge@example.net,48253XXXXXX,1963-06-05,XXXX-XXXX-9846,114-35-0580


In [ ]:
def mask_email(df, column_name):
    """
    Enmascara las direcciones de correo electrónico en la columna especificada del DataFrame,
    manteniendo solo el dominio.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a anonimizar.

    Returns:
        pd.DataFrame: El DataFrame modificado con los dominios de correo electrónico anonimizados.
    """
    try:
        # Extraer el dominio de correo electrónico y reemplazar la columna
        df.loc[:, column_name] = '****@' + df[column_name].str.split('@').str[1]
        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df


In [ ]:
df_anonymized = mask_email(df_anonymized, "email")

In [ ]:
df_anonymized.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Kimberly Burnett,****@example.net,59303XXXXXX,1990-08-21,XXXX-XXXX-5373,175-01-7801
1,Cheyenne Mendez,****@example.com,18879XXXXXX,1945-06-03,XXXX-XXXX-0644,483-80-4893
2,Francisco Williams,****@example.net,25335XXXXXX,1959-10-08,XXXX-XXXX-9100,091-36-2141
3,Rebecca Phillips,****@example.org,50130XXXXXX,1972-06-01,XXXX-XXXX-2014,754-41-3919
4,Andrea Jenkins,****@example.net,48253XXXXXX,1963-06-05,XXXX-XXXX-9846,114-35-0580


## Hashing

Una solución sencilla, al momento de trabajar con PII, es eliminar estos campos antes de compartir los datos. Sin embargo, en algunas ocasiones se necesita disponer de los datos de identificación personal. Por ejemplo, las empresas que comercializan servicios analizan que clientes tiene una probabilidad alta de cancelar su membresía o suscripción para poder ofrecerles descuentes o beneficios adicionales y retenerlos. En este caso, deben trabajar con información para identificar a cada cliente. Es posible anonimizar los campos con PII por medio de **hashing**.

El hashing es un proceso unidireccional de transformación de una cadena de caracteres de texto plano en una cadena única de longitud fija. El proceso de hashing tiene dos características importantes:
- Es muy difícil convertir un string "hasheado" en su forma original.
- La misma cadena de texto plano producirá el mismo resultado cifrado.

De esta forma, en vez de compartir campos con PII, vamos a compartir su versión "hasheada".

En el siguiente ejemplo, vamos a usar librería standard `hashlib` de Python.

In [ ]:
import hashlib
nombre = "Guido Franco"
nombre = nombre.lower()
hashlib.sha256(nombre.encode()).hexdigest()

'd615a5fc78a80330add0b5b84d09ba8ea4920138e1d746f0f5760e6b8767d76e'

In [ ]:
hashlib.sha256("GUIDO FRANCO".encode()).hexdigest()

'b8c657ad14b52b6f8dc6976e5408f527f102482f6b20414f9644466167081249'

In [ ]:
import hashlib

def hash_column(df, column_name):
    """
    Aplica el algoritmo "Hash" a los valores en una columna del DataFrame.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a hashear.

    Returns:
        pd.DataFrame: El DataFrame modificado con los valores hasheados en la columna especificada.
    """
    try:

        # Crear una nueva columna _hashed con los valores hasheados de la columna recibida
        df[f"{column_name}_hashed"] = df[column_name].apply(
            lambda value: hashlib.sha256(value.encode()).hexdigest()
            )

        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df

In [ ]:
#df_anonymized = hash_column(df_anonymized, "ssn")
df_anonymized = hash_column(df_anonymized, "full_name")

In [ ]:
df_anonymized.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn,ssn_hashed,full_name_hashed
0,Kimberly Burnett,****@example.net,59303XXXXXX,1990-08-21,XXXX-XXXX-5373,175-01-7801,c7a62be325180673b4ab783b3acca7a21aeb74175633fb...,cc292624de17f2cad653687b5479a5aa93b775476534f4...
1,Cheyenne Mendez,****@example.com,18879XXXXXX,1945-06-03,XXXX-XXXX-0644,483-80-4893,6f5f34183a9a6d4c244c08c50ca499cf550eaf75eb85e5...,ed0bcb127536c47f3a2b05e50ee57b1b5419f57a73ac3a...
2,Francisco Williams,****@example.net,25335XXXXXX,1959-10-08,XXXX-XXXX-9100,091-36-2141,147e5fc70e2c5c57d80049578e7f0ea16073bb39bbc4a0...,2ac9ddb7ba4a08333db46e60db12e3b7634b8ddc25a399...
3,Rebecca Phillips,****@example.org,50130XXXXXX,1972-06-01,XXXX-XXXX-2014,754-41-3919,9176dd94a10918ed52fb009cc09cde3b74d484fa22e8da...,799631a5843f8d0ab8d9c2a6f839db161167947ef4e9fa...
4,Andrea Jenkins,****@example.net,48253XXXXXX,1963-06-05,XXXX-XXXX-9846,114-35-0580,624f511960807f4f1500f0f9839e8f9610a7b1563bb97f...,0eabc1ae8d5cffffedbede49d5a46e1371cb3d3babdfe2...


In [ ]:
df_anonymized.drop(columns=["full_name", "ssn"], inplace=True)

In [ ]:
df_anonymized.head()

,email,phone_number,birth_date,credit_card_number,ssn_hashed,full_name_hashed
0,****@example.net,59303XXXXXX,1990-08-21,XXXX-XXXX-5373,c7a62be325180673b4ab783b3acca7a21aeb74175633fb...,cc292624de17f2cad653687b5479a5aa93b775476534f4...
1,****@example.com,18879XXXXXX,1945-06-03,XXXX-XXXX-0644,6f5f34183a9a6d4c244c08c50ca499cf550eaf75eb85e5...,ed0bcb127536c47f3a2b05e50ee57b1b5419f57a73ac3a...
2,****@example.net,25335XXXXXX,1959-10-08,XXXX-XXXX-9100,147e5fc70e2c5c57d80049578e7f0ea16073bb39bbc4a0...,2ac9ddb7ba4a08333db46e60db12e3b7634b8ddc25a399...
3,****@example.org,50130XXXXXX,1972-06-01,XXXX-XXXX-2014,9176dd94a10918ed52fb009cc09cde3b74d484fa22e8da...,799631a5843f8d0ab8d9c2a6f839db161167947ef4e9fa...
4,****@example.net,48253XXXXXX,1963-06-05,XXXX-XXXX-9846,624f511960807f4f1500f0f9839e8f9610a7b1563bb97f...,0eabc1ae8d5cffffedbede49d5a46e1371cb3d3babdfe2...


Se ha creado nuevas columnas: `ssn_hashed` y `full_name_hashed`. Es necesario borrar las columnas originales si se pretende compartir esta información, o bien reemplazarlo con su versión hasheada.

## Generalización
Hay ciertos elementos de datos que se relacionan más fácilmente con determinados individuos. Para protegerlos, utilizamos la generalización para **eliminar una parte** de los datos o **reemplazarlos por un valor común.**

A continuación, vamos a aplicar generalización sobre `birth_date`, eliminando el día y manteniendo solo el mes y el año.

> *La generalización nos permite lograr el k-anonimato (k-anonymity), un término estándar en la industria utilizado para describir una técnica para ocultar la identidad de individuos en un grupo de personas similares. En el anonimato k, k es un número que representa el tamaño de un grupo. Si para cualquier individuo del conjunto de datos, hay al menos k-1 individuos que tienen las mismas propiedades, entonces hemos conseguido el k-anonimato para el dataset. Por ejemplo, imaginemos un determinado dataset en el que k es igual a 50 y la propiedad es el código postal. Si observamos a cualquier persona dentro de ese conjunto de datos, siempre encontraremos a otras 49 con el mismo código postal. Por lo tanto, no podríamos identificar a ninguna persona sólo a partir de su código postal.*



In [ ]:
def truncate_date_to_month(df, column_name):
    """
    Trunca los valores en una columna de fechas del DataFrame hasta el mes.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna de fechas que se va a truncar.

    Returns:
        pd.DataFrame: El DataFrame modificado con los valores de la columna truncados al mes.
    """
    try:
        # Verificar si es un DataFrame
        if not isinstance(df, pd.DataFrame):
            raise ValueError("El argumento 'df' debe ser un DataFrame.")

        # Verificar si la columna existe
        if column_name not in df.columns:
            raise ValueError(f"La columna '{column_name}' no existe en el DataFrame.")

        # Verificar y convertir si es necesario
        if df[column_name].dtype != 'datetime64[ns]':
            try:
                df[column_name] = pd.to_datetime(df[column_name])
            except:
                raise ValueError(f"No se pudo convertir la columna '{column_name}' a tipo datetime.")

        # Truncar al mes
        df[column_name] = df[column_name].dt.to_period('Y')
        return df
    except Exception as e:
        print(f"Error: {e}")
        return df

In [ ]:
df_anonymized = truncate_date_to_month(df_anonymized, "birth_date")

In [ ]:
df_anonymized.head()

,email,phone_number,birth_date,credit_card_number,ssn_hashed,full_name_hashed
0,****@example.net,59303XXXXXX,1990,XXXX-XXXX-5373,c7a62be325180673b4ab783b3acca7a21aeb74175633fb...,cc292624de17f2cad653687b5479a5aa93b775476534f4...
1,****@example.com,18879XXXXXX,1945,XXXX-XXXX-0644,6f5f34183a9a6d4c244c08c50ca499cf550eaf75eb85e5...,ed0bcb127536c47f3a2b05e50ee57b1b5419f57a73ac3a...
2,****@example.net,25335XXXXXX,1959,XXXX-XXXX-9100,147e5fc70e2c5c57d80049578e7f0ea16073bb39bbc4a0...,2ac9ddb7ba4a08333db46e60db12e3b7634b8ddc25a399...
3,****@example.org,50130XXXXXX,1972,XXXX-XXXX-2014,9176dd94a10918ed52fb009cc09cde3b74d484fa22e8da...,799631a5843f8d0ab8d9c2a6f839db161167947ef4e9fa...
4,****@example.net,48253XXXXXX,1963,XXXX-XXXX-9846,624f511960807f4f1500f0f9839e8f9610a7b1563bb97f...,0eabc1ae8d5cffffedbede49d5a46e1371cb3d3babdfe2...


Hemos llegado al final, has conocido y aplicado algunas técnicas para proteger y anonimizar datos con PII.

In [ ]:
!mkdir -p datalake/silver/personal_data
df_anonymized.to_csv("datalake/silver/personal_data/people.csv", index=None)

Hagamos de cuenta que somos Data Scientists

In [ ]:
df_people = pd.read_csv(
    "datalake/silver/personal_data/people.csv",
    usecols=["full_name_hashed", "ssn_hashed"])
df_people.head()

,ssn_hashed,full_name_hashed
0,c7a62be325180673b4ab783b3acca7a21aeb74175633fb...,cc292624de17f2cad653687b5479a5aa93b775476534f4...
1,6f5f34183a9a6d4c244c08c50ca499cf550eaf75eb85e5...,ed0bcb127536c47f3a2b05e50ee57b1b5419f57a73ac3a...
2,147e5fc70e2c5c57d80049578e7f0ea16073bb39bbc4a0...,2ac9ddb7ba4a08333db46e60db12e3b7634b8ddc25a399...
3,9176dd94a10918ed52fb009cc09cde3b74d484fa22e8da...,799631a5843f8d0ab8d9c2a6f839db161167947ef4e9fa...
4,624f511960807f4f1500f0f9839e8f9610a7b1563bb97f...,0eabc1ae8d5cffffedbede49d5a46e1371cb3d3babdfe2...


In [ ]:
import numpy as np

df_people["churn_probability"] = np.random.uniform(0, 1, df_people.shape[0])

In [ ]:
df_people_churn = df_people[df_people["churn_probability"] >= 0.8]

In [ ]:
df_people_churn.head()

,ssn_hashed,full_name_hashed,churn_probability
1,6f5f34183a9a6d4c244c08c50ca499cf550eaf75eb85e5...,ed0bcb127536c47f3a2b05e50ee57b1b5419f57a73ac3a...,0.859364
5,0fe97cda887f4d6855a99eff1f7ea147cbefc299014bd2...,54178eb383f8bd7fdcfe823bde702a02418068862fff16...,0.814679
9,503a93ff5d9c1e001cdf4dfac279d17d5dafcb536f4331...,32a6084b3b8ff3a8c43d9ec86ebb7756c0c2d33822ad6b...,0.837985
10,1a796205239a023b45eb99f6005d1d0f451a6a998cd709...,cce391163de17a4148530124451c2f7d8d9785723f6d95...,0.921512
12,424736e9dc121696aae795fb1bda629f90499fa2f31bcb...,59a89753d7fb54d4bdeb39f154f812c00c7036297ba8dc...,0.940935


In [ ]:
df_people_sens = pd.read_csv("datalake/sensitive/personal_data/people.csv",
                             usecols=["full_name", "email", "phone_number", "ssn"])
df_people_sens.head()

,full_name,email,phone_number,ssn
0,Kimberly Burnett,aaronmartin@example.net,5930311660117,175-01-7801
1,Cheyenne Mendez,josephalejandro@example.com,1887903119443,483-80-4893
2,Francisco Williams,rodneygray@example.net,2533578229204,091-36-2141
3,Rebecca Phillips,davisbryan@example.org,5013047768596,754-41-3919
4,Andrea Jenkins,dhodge@example.net,4825387235845,114-35-0580


In [ ]:
df_people_sens = hash_column(df_people_sens.copy(), "ssn")
df_people_sens.head()

,full_name,email,phone_number,ssn,ssn_hashed
0,Kimberly Burnett,aaronmartin@example.net,5930311660117,175-01-7801,c7a62be325180673b4ab783b3acca7a21aeb74175633fb...
1,Cheyenne Mendez,josephalejandro@example.com,1887903119443,483-80-4893,6f5f34183a9a6d4c244c08c50ca499cf550eaf75eb85e5...
2,Francisco Williams,rodneygray@example.net,2533578229204,091-36-2141,147e5fc70e2c5c57d80049578e7f0ea16073bb39bbc4a0...
3,Rebecca Phillips,davisbryan@example.org,5013047768596,754-41-3919,9176dd94a10918ed52fb009cc09cde3b74d484fa22e8da...
4,Andrea Jenkins,dhodge@example.net,4825387235845,114-35-0580,624f511960807f4f1500f0f9839e8f9610a7b1563bb97f...


In [ ]:
pd.merge(df_people_churn, df_people_sens, on="ssn_hashed", how="inner")

,ssn_hashed,full_name_hashed,churn_probability,full_name,email,phone_number,ssn
0,85e0ada1b1a501cf2f26a184574395f8a4e88d5a9b74b0...,a00486ff0d1fb4f150b9887ea443b620d1c59375ede1ca...,0.887080,Amanda Price,justinquinn@example.com,1853295845131,655-50-3855
1,8eb7c32e3fba043970d97023137ee8b3245b95aefe8bb8...,e1d5b8ef6c927b0807d1d7fbcb574d926eea4037bf6e6d...,0.863922,Adrian Cole,ethancoffey@example.org,4020163823147,076-06-7050
2,a3e9a4e983bf55243fec09962ff37eb4641b0f61ca69a9...,f25cfab317f688e846d7ecb3788c9980aadd090e573644...,0.826983,Jenny Powers,williammarshall@example.org,1416233504380,071-25-6958
3,7f83f2842ca36552e4c78d35c92350fd151ba4f83279ca...,6a76bf69c36c307b6b2de05a7599a1e7f4f30a7e7b6804...,0.915311,Katherine Avila,tannerprice@example.com,6525310606709,714-21-4676
4,3a899540a4111273a62757c2c2930ef0349baa2da12532...,8c4367643e4c791c53da4453bc33449954533262737a86...,0.954656,Michael Wood,richardsullivan@example.org,7917350269142,865-86-0645
5,d42d67ea6dcecc343edd33255b489d70b5afa08459554b...,1e4b2da7a22cb7d5f9aac44d7fa00d9ae505051d7c565e...,0.919895,Paige Walker,perezmatthew@example.net,6179195443479,834-10-6805
6,af5082e8959eb29c9a4a88e7d322e83e6b61deb4db24c5...,519a3007a7652d02c21a97fc2d35aba5c9a17f05919729...,0.923977,Dana Hernandez,tuckersamantha@example.org,217070621238,033-26-8201
7,b37e8bfdd8a33ca2f40c20892fc64901d279e8bb2a1e4a...,132c831b5306a96b37249b496d0def2ea75a5f8187611b...,0.853481,Catherine Dixon,wrightmichael@example.net,4625189287657,183-15-5797
8,1df4e41e7af5d566f9b2bb726da206cdef2edaa077d1e3...,aa1552e134147f482a4bf5677fb3cf075a82297cfa947f...,0.822570,Daniel Lopez,larry11@example.com,6863681716295,150-93-9700
9,5d5fbb938f4037a1abbfaa2af02b3982de920a1f4aa220...,1939b20392dbafceb55153936f0a5009d59f3ccf2a3901...,0.990763,Elizabeth Mcgee,shane02@example.com,9735716226128,882-41-8536
